Create, Manipulate and Visualize Images with SampleData
==============================================================

This notebook is designed to demonstrates the elementary steps to store, manipulate and visualize **images** with the `SampleData` data platform class of the `pymicro.core` package. By **images**, it is intended volumic data that is defined on a *regular grid* of *voxels* (3D) or *pixels* (2D). 

Through this notebook, you will learn to:
- Transfer data from `numpy` arrays to an `image_object` from the `pymicro.core.image` package
- Create an Image group into a `SampleData` instance/file from an `image_object`
- Add scalar, vector and tensor Fields into 2D and 3D Image groups from `numpy` arrays
- Use Paraview to visualize the images

*** 

The `SampleData` class automatically synchronizes a **HDF5** and a **XDMF** file with each instance. The **XDMF** file serve a visualization purpose, as it is readable by Paraview, an advanced rederer for 2D visualization of volumic data. Through this Notebook, you will be indicated at various points to use Paraview to display the data that you will produced while executing its cells. To open a `SampleData` file in Paraview, simply open the software and simply use it to open the **XDMF**. At this moment, Paraview may require you to chose a specific reader to parse the **XDMF** file. At this point, chose `XDMF Reader`, and all the volumic datasets (images, meshes and their associated fields) will be displayed.  
<font color=red>WARNING: readers `Xdmf3ReaderS` and `Xdmf3ReaderT` do not properly parse `SampleData` files.</font>

#### Array ordering conventions

Throughout this Notebook, you will manipulate and visualize images, volumic data defined on regular grids. To display an image, the **XDMF** format provide Paraview with three informations:

1. The grid origin: coordinates of the first vertex of the voxels/pixels grid
2. The grid spacing: the size of voxels/pixels constituting the image
3. Data arrays: the values of the fields to display on the image grid of voxels/pixels

The correspondance between voxels/pixels and the data arrays elements is therefore implicit. Paraview interprets them in the descending order of the fastest varying indices, starting with the field element dimensionality, and then the grid dimensions. The last grid dimension is forced to be the $z$ coordinate. When considering scalar fields, the field dimensionality array dimension can be omitted. 

SampleData default settings use `C` array ordering, *i.e.* the fastest varying index for arrays is the last one, the rightmost one. That means, that arrays contained in SampleData images will be interpreted by paraview as follows:
- $\text{array}[Z,Y,Dim]$ in the case of vector or tensor 2D fields
- $\text{array}[Z,Y]$ in the case of scalar 2D fields
- $\text{array}[Z,Y,X,Dim]$ in the case of vector or tensor 3D fields
- $\text{array}[Z,Y,X]$ in the case of scalar 3D fields
 
***

## I - Prepare environment to use Sample Data (imports)

In [ ]:
import os
import math
import numpy as np
from pymicro.core.samples import SampleData
from pymicro.core.images import ImageObject
from matplotlib import pyplot as plt

## II - Create SampleData Instance
A `SampleData` instance is created with the filename *SD_images*. It will be synchronized with the files *SD_images.h5* and *SD_images.xdmf* in the same directory as this notebook. 

If you want extended information on the `SampleData` object manipulations, use the keyword argument `verbose=True` in the instanciation.

In order to suppress the *SD_images.h5* and *SD_images.xdmf* files at the deletion of the class instance (at the end of this Notebook), use the `autodelete=True` keyword argument. Using this option is recommended when a SampleData instance is used for temporary data manipulation, when learning to use SampleData or trying things.   

After creation of the `SampleData` instance, we print it to see the content of the dataset.

In [ ]:
sample = SampleData(filename='SD_images', verbose=True, autodelete=True)
print(sample)

## II - Creation of 2D images
First, we create a simple binary image of dimension 2 (pixelized fields), representing a square, by creating a `numpy` integer array. It will later be stored into `sample` into two different 2D images.

In [ ]:
image1 = np.zeros((25,25),dtype=int)
image1[5:20,5:20] = 1
plt.imshow(image1, cmap='jet')

## III - Creation of Image Objects 
We now create two `ImageObject` instances. This class is designed to represent a regular grid (2D or 3D) on which are defined several voxelized/pixelized fields (*i.e.* images). We create two objects of this class from `image1` two create 2 images with a different geometry but the same field content.

### First ImageObject
The default settings of ImageObject are `spacing=[1.,1.,1.]` and `origin=[0.,0.,0.]`, two attributes that control respectively the pixel (voxel in 2D) size and position of the first corner of the image. The first `ImageObject` is created with those default settings.
They define the geometry of the grid that will be stored latter in the SampleData object, and thus the position of the image in the visualization frame in Paraview.

Here, the image is defined through the shape of the `image1` array shape. It will thus be an image of *25x25* pixels. Given the aforementioned geometrical parameters, the grid will thus occupy the domain $0 \leq y \leq 25, 0 \leq z \leq 25$ when displayed in Paraview (see Paraview's array oredering conventions above).

In [ ]:
ImageObject_2D_1 = ImageObject(dimension=image1.shape) 
# add image1 as a field on the image object
ImageObject_2D_1.add_field(Field=image1, Field_name='test_image')

We can print the Image object content and attributes to check its content and geometry.

In [ ]:
print('Image dimension is',ImageObject_2D_1.dimension)
print('Image voxel size is',ImageObject_2D_1.spacing)
print('Image origin coordinates is',ImageObject_2D_1.origin)
plt.imshow(ImageObject_2D_1.fields['test_image'], cmap='jet')

### Second ImageObject
We now create a second object, with a pixel size twice as large as the first one, and an origin shifted in order to avoid overlap between the two images in the visualization frame. As the first image extends from 0 to 25 in the Y dimension, the origin of the second is set to 30. in this dimension. These settings are passed to the `ImageObject` constructor, as follows:

In [ ]:
ImageObject_2D_2 = ImageObject(dimension=image1.shape, origin=[0., 30.],
                       spacing=[2.,2.])
# add image1 as a field on the image object with a different name to avoid confusion
# later with Paraview visualization.
ImageObject_2D_2.add_field(Field=image1, Field_name='test_image_large')

We can print the Image object content and attributes to check its content and geometry.

In [ ]:
print('Image dimension is',ImageObject_2D_2.dimension)
print('Image voxel size is',ImageObject_2D_2.spacing)
print('Image origin coordinates is',ImageObject_2D_2.origin)
plt.imshow(ImageObject_2D_2.fields['test_image_large'], cmap='jet')

## IV - Add the Image Objects into SampleData instance

SampleData can directly store an `ImageObject` into a SampleData Image group. Such a group is a HDF5 Group that has attributes defining the geometry of the Image underlying grid, and children Nodes that contain the data array with values of the fields defined on the Image. 

To add an `ImageObject` into SampleData, simply use the `add_image` method, and specify identification metadata of the Image group. These metadata include the name of the HDF5 Group used to store the image, et the location where it is stored. Additionnally, an `indexname` is provided. SampleData allows to access data items with their names, pathes in the HDF5 tree, but also through index names or aliases, in most of the SampleData API methods. 

We start by adding the first `ImageObject`, and then print the content of the data set to verify it. Use of `replace=True` keyword argument is wise here. Indeed, you may execute the next cell many times while working through this Notebook. In doing so, you may end up asking SampleData to create an Image Groupy `TestIm2D` that already exists in the dataset. This is a forbidden operation, unless you specifically ask for it through the `replace=True` keyword argument. You may try to set `replace=False` in the next cell after creating the field node to observe that you will get an error in this case. 

In [ ]:
# Add first image object
sample.add_image(ImageObject_2D_1,imagename='TestIm2D', location='/',indexname='image2d', replace=True) 
print(sample)

In [ ]:
# Add second image object
sample.add_image(ImageObject_2D_2,imagename='TestIm2D_large',indexname='image2d_large', replace=True)

Now, two image groups named *TestIm2D* and *TestIm2D_large* have been created in the `SampleData` instance. We can print the content of the dataset again to verify it. You can see that the index of the dataset now includes both Image Groups and both image fields entries, and that the Groups and Fields HDF5 nodes information are printed: 

In [ ]:
print(sample)

## V - First visualization in Paraview
Now, we call the `pause_for_visualization` method. It flushes the data from the SampleData instance into the *SD_images.h5* and *SD_images.xdmf* files, and pauses the interpreter while closing the datafile objects. You can resume data management by pressing the `<Enter>` key, which will reopen the datafil objects. While the pause is on, you can open your files with Paraview for visualization (see next cell).  

In [ ]:
sample.pause_for_visualization()

Now, you can open the *SD_images.xdmf* in the Paraview software. Note that the XDMF file specifies the presence of 2 grids in the dataset, corresponding to the 2 SampleData image groups that have been created in the previous section. To get a nice visualization, you should open the file *SD_images.xdmf* twice in Paraview, that will give you 2 pipelines from the file, as illustrated below:
<img src="./pipelines.png" />

Then select only one image Group in the *Blocks* panel of the *Properties*  section of the Paraview Interface, like here:
<img src="./Property_block.png" />

You can now select the **Surface** render view mode, and to display the `'test_image'` and `'test_image_large'` data. What You should see in Paraview is similar to the image below:
<img src="./2DIm_view.png" />

You can see that the image sizes are consistent with what was set to construct the two `ImageObject` that have then been added into the SampleData instance. However, the fields are displayed in the $(Z,Y)$ plane, and not the $(X,Y)$ plane, because of the ordering convention used to interpret values order in Paraview.

You can now press the `<Enter>` key in the previous cell and continue your path through the Notebook.

## VI - 3D Images
Now that we have seen how the code works for simple 2D Images, we will now add similarly 2 3D Images to the dataset, from a single binary field. We start by creating the `numpy` arrays that contain the image field. We construct this time a "*sandwich*" binary field whose layers are normal to the X direction. To do this, we set all voxels in a specific $x$ range to 1, the other being left to 0.  

In [ ]:
image2 = np.zeros((25,25,25),dtype=int)
image2[:,:,5:20] = 1

Likewise, we create two `ImageObject` that will store two different 3D Images. We chose the voxel sizes so that each 2D Image has a 3D Image that has the same size in the (X,Y) plane, and the origins so that these 3D Images are in front of the corresponding 2D Image (corresponding in size). The dimensionality of the Image groups are automatically determined by SampleData by analyzing the `dimension` attribute of the `ImageObject` instance passed to the `add_image` method.

In [ ]:
# First ImageObject
# Default [1.,1.,1.] spacing is used, but origin is set to 10 unit of the corresponding [1.,1.]
# spacing 2D Image along Z.
ImageObject_3D_1 = ImageObject(dimension=image2.shape, origin=[0., 0., 10.]) 
# add image1 as a field on the image object
ImageObject_3D_1.add_field(Field=image2, Field_name='test_image_3D')

# Second ImageObject
# [2.,2.,2.] spacing is used, origin is set to -60 unit of the corresponding [2.,2.] spacing 2D Image along Z,
# in order to have the closest face at 10 units from the 2D Image.
ImageObject_3D_2 = ImageObject(dimension=image2.shape, spacing=[2.,2.,2.], origin=[0., 30., -60.]) 
# add image1 as a field on the image object
ImageObject_3D_2.add_field(Field=image2, Field_name='test_image_3D_large')

Likewise, we add these `ImageObject` instances into `sample` by using the `add_image` method.

In [ ]:
# Add first image object
sample.add_image(ImageObject_3D_1,imagename='TestIm3D', location='/',indexname='image3d', replace=True) 

# Add second image object
sample.add_image(ImageObject_3D_2,imagename='TestIm3D_large',indexname='image3d_large', replace=True)

# print dataset content
print(sample)

Now, we use `pause_for_visualization()` to flush the newly written image data into the HDF5 and XDMF files et allow visualization.

In [ ]:
sample.pause_for_visualization()

Now, open the XDMF file in Paraview 4 times to have 4 pipelines, display each image and the field it contains, and the grid axes. You should see something like this:
<img src="./3DIm_view.png" />

As you can see, the Renderview display properly the 4 images disposed as prescribed when creating the `ImageObject` instances. Note that the 2D Images are displayed in the plane $x=0$. **This is a strong constraint introduced by the XDMF data format.** For more advanced 2D Image rendering, you should use Mesh groups of regular square elements, and fields defined at each element center. 

You can now press the `<Enter>` key in the previous cell and continue your path through the Notebook.

## VII - Store Vector and Tensor fields
Now that we are familiar with the manipulation and visualization of scalar image fields, we will move to fields with a higher dimensionality, vector and tensor fields. We will see by the same occasion, how to add fields to an already existing image Group in a `SampleData` instance.

We start by creating a 2D vector field, that will be stored on the large 2D image. We want, for this example, a vector field that is normal to the 2D image plane (*i.e* parallel to the $x$ direction), and that has opposites values on the two parts of the binary field already stored on the 2D image.

In [ ]:
# initialize vector field
vect_field_2D = np.zeros((25,25,3))
# We create a field with a value that depends on the binary field value of the 2D image
# First we set one value
vect_field_2D[:,:,:] = [5., 0., 0.]
# Then we get the field from the image
bin_field = sample.get_node(name='test_image_large')
# And finally we change the field value on the central square
vect_field_2D[np.where(bin_field)] = [-5., 0., 0.]
# check content
print('Field shape is',vect_field_2D.shape)
plt.imshow(vect_field_2D[:,:,0])

Use the method `add_data_array` to add a field to the wanted image. Here we chose to add it on the large 2D image. It can be designated through:
- it's path : `'/TestIm2D_large'`
- it's name : `'TestIm2D_large'` (close to its path in this case)
- it's indexname : `'image2d_large'`
You can use the method `print_index` to access these informations, as follows:

In [ ]:
sample.print_index()

Now, you can use `add_data_array` with the designation name you chose for the Image Group where you want to add your field (here `'image2d_large'`). Like before with the addition of an Image Group, use of `replace=True` keyword argument is wise here. Indeed, you may execute the next cell many times while working through this Notebook. In doing so, you may end up asking SampleData to create a data array Node that already exists in the dataset. This is a forbidden operation, unless you specifically ask for it through the `replace=True` keyword argument. You may try to set `replace=False` in the next cell after creating the field node to observe that you will get an error in this case. 

You can now press the `<Enter>` key in the previous cell and continue your path through the Notebook.

Once the field is added, we can print the image Group content to ensure that it has been added. Then, we use again the `sample.pause_for_visualization()` method to flush data into files and open it into Paraview.

In [ ]:
sample.add_data_array(location='image2d_large',name='vector_field_2D', array=vect_field_2D, replace=True)
sample.print_group_content('image2d_large')
sample.pause_for_visualization()

Next, we open the XDMF file in Paraview, select the `TestIm2D_large` block, and the `vector_field_2D` data field for coloring. We can now plot the vector coordinate fields, as follows:
<img src="./Vector_2D.png" width="750"/>

We can also apply the Glyph filter to the `TestIm2D_large` block to display the vector field. After suitable coloring of arrows, you can get this kind of visualization:
<img src="./Vector_2D_glyph.png" width="750"/>

You can now press the `<Enter>` key in the previous cell and continue your path through the Notebook.

We can do the same things with a vector field stored on a 3D image. This time, we create a vector field with three slices in which the field points toward one grid direction. The slices are stacked along the $x$ direction (so they need to be defined by a range within the third index of the array): 

In [ ]:
# initialize vector field
vect_field_3D = np.zeros((25,25,25,3))
# We create an homogeneous vector field
vect_field_3D[:,:,0:10,:] = [0., 0., 1.]
vect_field_3D[:,:,10:15,:] = [0., 1., 0.]
vect_field_3D[:,:,15:,:] = [1., 0., 0.]
print('Field shape is',vect_field_3D.shape)

We add it in the dataset:

In [ ]:
sample.add_data_array(location='image3d_large',name='vector_field_3D', array=vect_field_3D, replace=True)
sample.print_group_content('image3d_large')
sample.pause_for_visualization()


And visualize it in Paraview:
<img src="./vect_3D.png" width="750"/>

Here, we have used 2 RenderView. One (right image) displays a `Surface` representation of the `'vector_field_3D'` data in the `'image3d_large'` Block (see Paraview's panels). The other (left image) displays the same dataset after applying the `Glyph` filter (and some fine tuning aspect options), that allow to graphically plot vector fields, with arrows for instance. 

As you can see, the field looks fine and is consistent with what we defined, once the Paraview array ordering convention is ackowledged. 

You can now press the `<Enter>` key in the previous cell and continue your path through the Notebook.

Finally, we will add a symmetric tensor field stored on the 3D image. We will create a tensor field with three non zeros and heterogeneous axial components, defined by three mathematical functions of the corresponding axial grid coordinate. 

Paraview considers tensor fields, as 9 components vector fields. Hence, an array that contains 9 components tensor field data should be of shape $(N_x,N_y,N_z,9)$ (in 2D) or $(N_x,N_y,N_z,9)$ (in 3D). Along the last dimension, Paraview considers the elements of the array with the ordering convention below:

$$ T = 
\begin{bmatrix}
T_{xx} & T_{xy} & T_{xz}\\
T_{yx} & T_{yy} & T_{yz}\\
T_{zx} & T_{zy} & T_{zz}
\end{bmatrix}
=
\begin{bmatrix}
T[0] & T[1] & T[2]\\
T[3] & T[4] & T[5]\\
T[6] & T[7] & T[8]
\end{bmatrix} $$

where $T$ is one vector array that correspond to a $[i,j,k,:]$ slice of the tensor field.

On the other hard, symmetric tensors are considered as a 6 components vector fields. For them, Paraview considers ordering as follows:

$$ T = 
\begin{bmatrix}
T_{xx} & T_{xy} & T_{xz}\\
T_{yx} & T_{yy} & T_{yz}\\
T_{zx} & T_{zy} & T_{zz}
\end{bmatrix}
=
\begin{bmatrix}
T[0] & T[1] & T[2]\\
T[1] & T[3] & T[4]\\
T[2] & T[4] & T[5]
\end{bmatrix} $$

These orderings indicate how the last dimension of the `numpy` that we will create to store the field will be interpreted by the Renderer. Let's create a symmetric tensor field:

In [ ]:
# initialize a symmetric tensor field numpy array with shape (Nz,Ny,Nx,6)
tens_field_3D = np.zeros((25,25,25,6))
# We create a zz component that is the square of the z coordinate
for i in range(25):
    tens_field_3D[i,:,:,5] = (i/24.)**2.
# We create a yy component that is the square root of the y coordinate
for j in range(25):
    tens_field_3D[:,j,:,3] = (j/24.)**(1./2.)
# We create a xx component that is the sine of the x coordinate (half period)
for k in range(25):
    tens_field_3D[:,:,k,0] = math.sin(math.pi*(k/24.))
print('Field shape is',tens_field_3D.shape)

Now, once again, we add the tensor field in the dataset, check the 3D Image group content, and call the `sync` method in order to check out the result in Paraview while leaving the file opened by Python (in this Notebook's interpreter !).

In [ ]:
sample.add_data_array(location='image3d_large',name='tensor_field_3D', indexname='symtens',
                      array=tens_field_3D, replace=True)
sample.print_group_content('image3d_large')
sample.pause_for_visualization()


Then, we open the XDMF file in Paraview, select the `TestIm3D_large` block, and the `tensor_field_3D` data field for coloring, and we are able to plot the tensor field components, as below:
<img src="./Tensor_3D.png" width="1000"/>

You can now press the `<Enter>` key in the previous cell and continue your path through the Notebook.

## THE END -- Suppress SampleData instance
Now that we have finished using our `SampleData`, we can delete the instance that we used. If you created it with the `autodelete=True` keyword argument, then this will erase the files created by the class to store the data. 

In [ ]:
del sample

## To go further

You will find in this directory a *SD_images_ref.h5* and a *SD_images_ref.xdmf* files, created with this notebook as a reference output. You can play with the various parameters of the methods in the file and see how it changes how images display in Paraview, and compare it to this reference file. You can also explore how to compress these images, but a specific Notebook will be dedicated to this aspect of using SampleData.

Thank you for reaeding this up to the end !